In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2303, -0.3061, -0.1271,  0.1772,  0.2023, -0.2959, -0.0432, -0.1557,
          0.1620, -0.1070],
        [ 0.1807, -0.3342, -0.0825,  0.2948,  0.0989, -0.3278, -0.0933, -0.0665,
          0.1172, -0.1688]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
# 实例化类
net = MLP()
net(X)

tensor([[-0.0590, -0.1690,  0.1156,  0.1075,  0.0334, -0.0009, -0.1247, -0.1174,
         -0.0943,  0.0129],
        [ 0.0219, -0.2285,  0.0439, -0.0196, -0.0426, -0.0227, -0.1940, -0.0690,
          0.0035,  0.0513]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for index, module in enumerate(args):
            # 这里，module是Module类的一个实例，保存在Module类的成员变量modules中
            self._modules[str(index)] = module

    def forward(self, X):
        # orderdDict保证了按照成员添加的顺序遍历他们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3698, -0.0881, -0.0695,  0.0899, -0.0168,  0.0517, -0.2054, -0.5365,
          0.3500,  0.2388],
        [ 0.3607, -0.0190, -0.0240,  0.1017,  0.0664,  0.0579, -0.1900, -0.2866,
          0.3188,  0.1924]], grad_fn=<AddmmBackward0>)

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 指定requires_grad=False不计算梯度的随机全中参数，因此在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        # print('init_rand_weight:',self.rand_weight)
        self.linear = nn.Linear(20, 20)

    # 可以自定义在前向计算中采用
    def forward(self, X):
        # 放入线性层计算
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0838, grad_fn=<SumBackward0>)

In [12]:
# 混合搭配使用各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1966, grad_fn=<SumBackward0>)